In [14]:
%pip install -q kfp google-cloud google-cloud-bigquery google-cloud-storage google-cloud-aiplatform db_dtypes google_cloud_pipeline_components pandas_gbq

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.11.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.3 which is incompatible.
kfp-pipeline-spec 0.6.0 requires protobuf<5,>=4.21.1, but you have protobuf 5.29.3 which is incompatible.


In [1]:
import pandas as pd
import os
from datetime import datetime
import pytz as tz

In [1]:
#!gcloud auth login
#!gcloud auth application-default login

In [10]:
from google.cloud import bigquery

client = bigquery.Client()
project_id = 'project-mlops9-cm'
dataset_id='coches'
folder_path = 'data/input'

**Carga de CSVs a BigQuery**

In [7]:
dataser_ref = client.dataset(dataset_id)

for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):  ## para detectar si la ultima cadena del nombre tenga .csv
        table_id = os.path.splitext(file_name)[0]
        table_ref = dataser_ref.table(table_id)
        
        job_config = bigquery.LoadJobConfig(
            source_format = bigquery.SourceFormat.CSV,
            skip_leading_rows = 1,
            autodetect = True,
        )
        
        #data/pipeline_train_model_input/filename.csv
        csv_file_path = os.path.join(folder_path, file_name)
        
        with open(csv_file_path, 'rb') as source_file:
            load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
            
        print(f'Starting job {load_job.job_id} for {file_name}')
        
        load_job.result() # para indicar q tiene q esperar q el job de carga finalice
        
        destination_table = client.get_table(table_ref)
        print(f'loaded table {destination_table.num_rows} rows into {table_ref}')



Starting job 1d19aa84-3e0b-40d8-8c4d-c40e53015448 for selected_features.csv
loaded table 16 rows into project-mlops9-cm.coches.selected_features
Starting job 09aba5db-6d02-406e-8da0-c0ae1953ed3a for test_coche.csv
loaded table 822 rows into project-mlops9-cm.coches.test_coche
Starting job 4095c786-627a-4a6e-a1ac-377fdc8906dd for train_coche.csv
loaded table 3284 rows into project-mlops9-cm.coches.train_coche


In [3]:
import kfp
from google.cloud import aiplatform
from kfp.v2 import dsl, compiler
from kfp.v2.dsl import (component, Input, Output, Dataset)
from typing import NamedTuple
from google_cloud_pipeline_components.v1.vertex_notification_email import VertexNotificationEmailOp

C:\Users\cmatos\AppData\Local\Temp\ipykernel_16224\2901874434.py:3: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from kfp.v2 import dsl, compiler


In [65]:
@component(packages_to_install=["google-cloud-bigquery"])
def validate_data(data_table: str,) -> NamedTuple("Outputs",[("condition", str)],):
    from google.cloud import bigquery

    client = bigquery.Client()

    try:
        client.get_table(data_table)
        condition = "true"
    except Exception as e:
        condition = "false"

    return (condition,)

In [5]:
@component
def error_op(msg: str):
    raise(msg)

In [63]:
@component(packages_to_install=["pandas", "google-cloud-bigquery",
                               "google-cloud-bigquery-storage",
                               "db_dtypes",
                               "numpy",
                               "sckikit-learn",
                               "pyarrow",
                               "pandas_gbq"])
def process_data(project: str, 
                 data_table: str,
                 dataset: Output[Dataset]):
    from google.cloud import bigquery
    import pandas as pd
    import numpy as np
    import pyarrow.parquet as pq
    
    
    client = bigquery.Client(project)
    
    data = client.query(
    '''SELECT * FROM `{dsource_table}`
    '''.format(dsource_table = data_table)
    ).to_dataframe()
        
    data["running"] = data["running"].astype(str)
    data["running"] = data["running"].str.replace(" km","")
    data["running"] = data["running"].str.replace(" miles","")
    data["running"] = data["running"].astype(float)
    data["running"]

    # Variables temporales
    TEMPORAL_VARS = ['year']
    REF_VAR = datetime.now().year
    data[TEMPORAL_VARS] = REF_VAR - data[TEMPORAL_VARS]

    data.to_parquet(f'{dataset.path}.parquet', engine='pyarrow', index=False)
    
    

c:\Users\cmatos\AppData\Local\anaconda3\envs\env-mlops\lib\site-packages\kfp\dsl\component_decorator.py:126: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.9' to 'python:3.10' on Oct 1, 2025. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.10.
  return component_factory.create_component_from_func(


In [58]:
@component(packages_to_install=["pandas", "google-cloud-bigquery",
                               "google-cloud-bigquery-storage",
                               "db_dtypes",
                               "pyarrow",
                               "joblib",
                               "sckikit-learn",
                               "numpy"])
def train_model(project: str, 
                features_table: str,
                inputd: Input[Dataset],
                X_train_t: Output[Dataset]):
    from google.cloud import bigquery
    from google.cloud import storage
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import Lasso
    
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import MinMaxScaler
    from feature_engine.transformation import YeoJohnsonTransformer
    from feature_engine.encoding import OrdinalEncoder
    from sklearn.model_selection import train_test_split
    import joblib
    
    client = bigquery.Client(project=project)
    
    data =pd.read_parquet(f'{inputd.path}'.parquet)
    #Y_train = client.query(
    #'''SELECT * FROM `{dsource_table}`
    #'''.format(dsource_table = source_y_train_table)
    #).to_dataframe()

    features = client.query(
    '''SELECT * FROM `{dsource_table}`
    '''.format(dsource_table = features_table)
    ).to_dataframe()

    
    features = features['string_field_0'].tolist()
    
    # Separamos dataset en train y test
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['price'], axis=1), # predictive variables
        data['price'], # target
        test_size=0.1, # portion of dataset to allocate to test set
        random_state=0, # we are setting the seed here
    )
    
    # Transformación de variable target
    y_train = np.log(y_train)
    y_test = np.log(y_test)
    
    # Variables seleccionadas del proceso feature selection
    FEATURES = features

    X_train = X_train[FEATURES]
    X_test = X_test[FEATURES]
    
    # Variables para tranformaciion yeo 
    NUMERICALS_YEO_VARS = ['running', 'motor_volume', 'year']

    # Variables categoricas para codificar
    CATEGORICAL_VARS = ['model', 'motor_type', 'color', 'type', 'status']


    X_train[CATEGORICAL_VARS] = X_train[CATEGORICAL_VARS].astype('category')

    pipeline = Pipeline([
    # VARIABLE TRANSFORMATION
     ('yeojohnson', YeoJohnsonTransformer(variables=NUMERICALS_YEO_VARS)),
     #('log', LogTransformer(variables=NUMERICALS_YEO_VARS)),
        
     # CATEGORICAL ENCODING
    ('categorical_encoder', OrdinalEncoder(
        encoding_method='ordered', variables=CATEGORICAL_VARS)),
        
    # ESCALAMIENTO
    ('scaler', MinMaxScaler()),

    # MODELO
    ('Lasso', Lasso(alpha=0.001, random_state=0)),
    ])

    pipeline.fit(X_train, y_train)
    
    
    pipeline_file_name = '/data/model/precio_coches_pipeline.joblib'
    joblib.dump(pipeline, pipeline_file_name)

    
    # uplead to cs}
    storage_client = storage.Client(project = project)
    bucket_name = "cmatos_bk"
    destination_blob_name ="model/precio_coches_pipeline.joblib"
    
    X_train.to_parquet(f'{X_train_t.path}.parquet', engine='pyarrow', index=False)
    
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(pipeline_file_name)

In [59]:
@component(
    packages_to_install=[
        "google-cloud-bigquery",
        "google-cloud-bigquery-storage",
        "pandas",
        "joblib",
        "db-dtypes",
        "pyarrow",
        "pandas-gbq",
        "google-cloud-storage",
        "pytz"
    ],
)
def prediction(
    project: str,
    table_id: str,
    path_model: str,
    input_X_train: Input[Dataset],
):  
    import sys
    from datetime import datetime
    import pandas as pd
    from google.cloud import bigquery
    from google.auth import default
    import pandas_gbq
    from google.cloud import storage
    from joblib import load
    from io import BytesIO
    from pytz import timezone

    import pandas as pd
    import numpy as np
    import joblib
    from datetime import datetime

    
    X_train =pd.read_parquet(f'{input_X_train.path}'.parquet)

    TZ = timezone('America/Lima')
    FORMAT_DATE = "%Y-%m-%d"
    
    # Cliente BigQuery
    client = bigquery.Client(project=project)
    

    def generate_datetime_created():
        return datetime.now()
    
    def generate_date_created():
        return datetime.now(TZ).date().strftime(FORMAT_DATE)
    
    
    def load_model_from_gcs(path_model):
        # Inicializar el cliente de Cloud Storage
        storage_client = storage.Client()

        # Obtener el nombre del bucket y la ruta del objeto
        bucket_name, blob_name = path_model.replace("gs://", "").split("/", 1)

        # Obtener el objeto desde Cloud Storage
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        model_bytes = blob.download_as_string()

        # Cargar el modelo desde los bytes obtenidos
        model = load(BytesIO(model_bytes))

        return model

    model = load_model_from_gcs(path_model)

    # Realizar la predicción
    predictions = model.predict(X_train)
    predicciones_sescalar = np.exp(predictions)
    predictions = pd.DataFrame(predicciones_sescalar, columns=['prediction'])

    # Obtener el user_id de la sesión actual en BigQuery
    user_id = client.query("SELECT SESSION_USER()").to_dataframe().iloc[0, 0]

    # Agregar campos de auditoría
    start_time = generate_datetime_created()
    execute_date = generate_date_created()
    
    predictions['creation_user'] = user_id
    predictions['process_date'] = datetime.strptime(execute_date, '%Y-%m-%d')
    predictions['process_date'] = pd.to_datetime(predictions['process_date']).dt.date
    predictions['load_date'] = pd.to_datetime(start_time)
    
    # Guardar el resultado en BigQuery 
    pandas_gbq.to_gbq(predictions , table_id, if_exists='append', project_id=project)

    print("Predicción generada y guardada en BigQuery.")

In [66]:

@kfp.dsl.pipeline(name="pipeline_m5",
                  description="intro",
                  pipeline_root="gs://cmatos_bk/demo")
def main_pipeline(
    project: str,
    data_table: str,
    features_table: str,
    table_id: str,
    path_model: str,
    gcp_region: str = "us-central1",
):
    
    notify_email_task = VertexNotificationEmailOp(recipients=["jchristian.mp@gmail.com"])
    notify_email_task.set_display_name('Notification Email')
    
    with dsl.ExitHandler(notify_email_task, name="Execute pipeline prediction"):

        validate_tables_job = validate_data(
            data_table = data_table
        )
        validate_tables_job.set_display_name('Validate Data')

        with dsl.Condition(
            validate_tables_job.outputs['condition']=="false",
            name="no-execute",
        ):
            error_op(msg="No se logro validar las tablas de ingesta.")


        with dsl.Condition(
            validate_tables_job.outputs['condition']=="true",
            name="execute",
        ):
  
            process_dataset = process_data(
                project=project,
                data_table=data_table                
            )
            process_dataset.set_display_name("Process_Data")

            train = train_model(
                project=project,
                features_table=features_table,
                inputd=process_dataset.output
            ).after(process_dataset)
            train.set_display_name("Train_Model")

            prediction_model = prediction(
                project = project,
                table_id = table_id,
                path_model = path_model,
                input_X_train=train.output
            ).after(train)
            prediction_model.set_display_name("Prediction_Model")

C:\Users\cmatos\AppData\Local\Temp\ipykernel_16224\673917678.py:23: DeprecationWarning: dsl.Condition is deprecated. Please use dsl.If instead.
  with dsl.Condition(
C:\Users\cmatos\AppData\Local\Temp\ipykernel_16224\673917678.py:30: DeprecationWarning: dsl.Condition is deprecated. Please use dsl.If instead.
  with dsl.Condition(


In [67]:
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=main_pipeline,
    package_path="pipelinemodel_m5.json"
)

In [68]:
from google.cloud import storage

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"Archivo {source_file_name} subido a {destination_blob_name} en el bucket {bucket_name}.")

# Define las variables
bucket_name = "cmatos_bk"
destination_blob_name = "demo/pipelinemodel_m5.json"
pipeline_file = "pipelinemodel_m5.json"
# Llamar a la función para subir el archivo
upload_to_gcs(bucket_name, pipeline_file, destination_blob_name)

Archivo pipelinemodel_m5.json subido a demo/pipelinemodel_m5.json en el bucket cmatos_bk.


In [69]:
aiplatform.init(project="project-mlops9-cm", location="us-central1")

In [71]:
job = aiplatform.PipelineJob(
    display_name="pipeline coches",
    template_path="gs://cmatos_bk/demo/pipelinemodel_m5.json",
    #job_id="mlops72",
    enable_caching=False,
    project="project-mlops9-cm",
    location="us-central1",
    parameter_values={"project": "project-mlops9-cm", 
                      "data_table": "project-mlops9-cm.coches.train_coche",
                      "features_table": "project-mlops9-cm.coches.selected_features",
                      "table_id": "project-mlops9-cm.coches.predictions",
                      "path_model": "gs://cmatos_bk/model/precio_coches_pipeline.joblib"
                     }
)

print('submit pipeline job ...')
job.submit(service_account="<<account>>@project-mlops9-cm.iam.gserviceaccount.com")

submit pipeline job ...
Creating PipelineJob
PipelineJob created. Resource name: projects/129100651954/locations/us-central1/pipelineJobs/pipeline-m5-20250115015233
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/129100651954/locations/us-central1/pipelineJobs/pipeline-m5-20250115015233')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/pipeline-m5-20250115015233?project=129100651954
